# Dataset

In [1]:
import kagglehub
from pathlib import Path
import os
# Download latest version

ds_path = kagglehub.dataset_download("alsaniipe/chest-x-ray-image")
ds_path

/home/pigmorais/NN/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/pigmorais/.cache/kagglehub/datasets/alsaniipe/chest-x-ray-image/versions/1'

In [2]:
data_path = os.path.join(ds_path,"Data/")
train_path = os.path.join(data_path, "train/")
test_path = os.path.join(data_path, "test/")

In [3]:
from PIL import Image
import torch
import torchvision.transforms.v2 as v2

def load_img(path):
  # Le a imagem em diversos formatos e garante que a imagem tenha 3 canais
  img = Image.open(path).convert('RGB')
  # converte para um tensor do pytorch
  img = v2.functional.to_image(img)
  # garante que seja uma imagem de 8 bits reescalando os valores adequadamente
  img = v2.functional.to_dtype(img, dtype=torch.uint8, scale=True) # Remove this line
  return img # Return as PIL Image

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, IterableDataset
import torch.optim as optim
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from pathlib import Path # Import Path

class ChestXrayDataset(IterableDataset):
  def __init__(self, data_path, transforms):
    self._data_path = data_path
    self._transforms = transforms

  def __iter__(self):
    class_dirs = sorted([d for d in Path(self._data_path).iterdir() if d.is_dir()]) # Convert to Path
    class_iters = [iter(d.iterdir()) for d in class_dirs]

    # balanced batch
    while True:
      batch = []
      all_done = True


      for class_idx, class_iter in enumerate(class_iters):
        class_batch = []
        try:
          for _ in range(self._batch_size):
            path = next(class_iter)
            sample = self._preprocess(path)
            class_batch.append(sample)
          batch.extend(class_batch)
          all_done = False
        except StopIteration:
          continue # class folder ended

      if batch:
        yield batch
      if all_done:
        break

  def _preprocess(self, path):
    img = load_img(path)
    return (self._transforms(img), self._get_label_from_path(path))

  def _get_label_from_path(self, path):
    _path = str(path).lower() # Convert path to string for lower()
    if 'covid' in _path: return 0
    elif 'normal' in _path: return 1
    else: return 2  # pneumonia

In [ ]:
from torchvision import transforms

train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
train_data = ChestXrayDataset(data_path=train_path, transforms=train_transforms)
train_data_loader = DataLoader(dataset=train_data, batch_size=64, num_workers=2)

val_data = ChestXrayDataset()

test_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


test_data = ChestXrayDataset(data_path=test_path, transforms=test_transforms)
test_data_loader = DataLoader(dataset=test_data, batch_size=64 num_workers=2)

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

def train_valid_split_idx(batch_size=None, n_classes=3, train_size=0.8):
  train = list()
  valid = list()
  if train_size > 1.0 or train_size <= 0:
    train_size = 0.8

  for _ in range(n_classes):
    i = 0
    j = i + (batch_size//n_classes)
    idx = list(range(i, j))
    np.random.shuffle(idx)
    _end = int(train_size * len(idx))
    train.extend(idx[:_end])
    valid.extend(idx[_end:])

  train = np.asarray(train)
  valid = np.asarray(valid)

  return train, valid


def save_checkpoint(epoch, checkpoint_dir, state_dict, save_every_nth_epoch=2):
  if (epoch + 1) % save_every_nth_epoch == 0:
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith('model_checkpoint') and f.endswith('.pth')]
    checkpoints = sorted(checkpoints, key=lambda f: os.path.getmtime(os.path.join(checkpoint_dir, f)))

    if len(checkpoints) >= 3:
      oldest = checkpoints[0]
      os.remove(os.path.join(checkpoint_dir, oldest))

    new_index = epoch + 1
    checkpoint_path = os.path.join(checkpoint_dir, f'model_checkpoint{new_index}.pth')
    torch.save(state_dict, checkpoint_path)

def train_model(model=None, optimizer=None, train_data_loader=None, loss_module=None, epochs=200, logging_dir='drive/MyDrive/Experimentos/runs/cnn'):
  model.train()
  device = next(model.parameters()).device
  #writer = SummaryWriter(logging_dir)

  for epoch in tqdm(range(epochs)):
    epoch_loss = []
    val_loss = []

    # OBS: validação por batch ao invés de ser por epoch (pior, sim, mas é um teste válido caso seu dataset seja IMENSO (mundo real) )

    for batch in train_data_loader:
      # split train and validation images
      #inputs, labels = map(list, zip(*batch)) # Remove this line
      inputs, labels = zip(*batch) # Unpack the batch into images and labels
      # Reshape inputs to flatten the first two dimensions (DataLoader batch and Dataset batch)
      inputs = torch.cat(inputs, dim=0).to(device) # Concatenate the tensors in the list along the batch dimension
      labels = torch.cat(labels, dim=0).to(device) # Concatenate the labels as well

      # inputs = torch.stack(inputs).numpy() # Remove this line
      # labels = np.array(labels) # Remove this line

      train_idx, val_idx = train_valid_split_idx(len(inputs)) # Use the total number of samples in the combined batch
      inputs_train, labels_train, inputs_val, val_labels  = inputs[train_idx], labels[train_idx], inputs[val_idx], labels[val_idx]

      ##########################################################################
      #training mini-batch
      optimizer.zero_grad()
      #outputs = model(inputs) # Remove this line
      outputs = model(inputs_train) # Use inputs_train
      loss = loss_module(outputs, labels_train.long()) # Use labels_train and convert to long
      loss.backward()
      optimizer.step()
      epoch_loss.append(loss.item())
      ##########################################################################
      # validation mini-batch
      with torch.no_grad():
        #outputs = model(inputs_val) # Remove this line
        outputs = model(inputs_val) # Use inputs_val
        loss = loss_module(outputs, val_labels.long()) # Use val_labels and convert to long
        val_loss.append(loss.item())

      save_checkpoint(epoch=epoch, checkpoint_dir='drive/MyDrive/Experimentos/NN/', state_dict=model.state_dict(), save_every_nth_epoch=10)
      print(f'epoch {epoch} train loss: {epoch_loss[-1]} val loss {val_loss[-1]}')
    



    #print(train_imgs.shape) # (153, 1, 3, 224, 224)
    #print(valid_imgs.shape) # (39, 1, 3, 224, 224)

In [ ]:
# feature extractor
conv = nn.Sequential(
    nn.Conv2d(3, 6, 5), # Changed input channels from 1 to 3
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(6, 16, 5),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Flatten(),   # lembre-se do flatten!
    nn.Linear(16 * 53 * 53, 120), # Updated input size to 16 * 53 * 53
    nn.ReLU(),
    nn.Linear(120, 84)
)

# rede completa: feature extractor + classifier
net = nn.Sequential(
    conv,
    nn.ReLU(),
    nn.Linear(84, 3) # Changed output size to 3 for 3 classes
)

In [ ]:
import torch.optim as optim

loss_module = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)
train_model(model=net, optimizer=optimizer, train_data_loader=train_data_loader, loss_module=loss_module, epochs=3)

  0%|          | 0/3 [00:00<?, ?it/s]

epoch 0 train loss: 1.1151663064956665 val loss 0.8107205629348755
epoch 0 train loss: 0.8001296520233154 val loss 0.32571619749069214
epoch 0 train loss: 0.3250850439071655 val loss 0.0562884695827961
epoch 0 train loss: 4.853396415710449 val loss 3.096543550491333
epoch 0 train loss: 3.110407590866089 val loss 1.6809492111206055
epoch 0 train loss: 1.6682136058807373 val loss 0.9250344038009644
epoch 0 train loss: 0.9239198565483093 val loss 0.6195053458213806
epoch 0 train loss: 0.623062014579773 val loss 0.49903732538223267
epoch 0 train loss: 0.4928534924983978 val loss 0.4165429174900055
epoch 0 train loss: 1.9989428520202637 val loss 1.8669449090957642
epoch 0 train loss: 1.8672266006469727 val loss 1.7214477062225342
epoch 0 train loss: 1.6730124950408936 val loss 1.5152573585510254
epoch 0 train loss: 1.5205094814300537 val loss 1.3460578918457031
epoch 0 train loss: 1.3525410890579224 val loss 1.2316302061080933
epoch 0 train loss: 1.2383354902267456 val loss 1.13220500946044

 33%|███▎      | 1/3 [02:16<04:33, 136.50s/it]

epoch 1 train loss: 1.9947657585144043 val loss 1.943920373916626
epoch 1 train loss: 1.993330955505371 val loss 1.8124158382415771
epoch 1 train loss: 1.8429354429244995 val loss 1.685805082321167
epoch 1 train loss: 1.1686718463897705 val loss 1.1699211597442627
epoch 1 train loss: 1.169166088104248 val loss 1.1692399978637695
epoch 1 train loss: 1.169073462486267 val loss 1.167342185974121
epoch 1 train loss: 1.1671391725540161 val loss 1.1644409894943237
epoch 1 train loss: 1.1645970344543457 val loss 1.1611661911010742
epoch 1 train loss: 1.1615129709243774 val loss 1.157813310623169
epoch 1 train loss: 0.8455618619918823 val loss 0.8493484258651733
epoch 1 train loss: 0.8470097780227661 val loss 0.8383827209472656
epoch 1 train loss: 0.8321794271469116 val loss 0.8169097900390625
epoch 1 train loss: 0.8112730383872986 val loss 0.7965784668922424
epoch 1 train loss: 0.7793148159980774 val loss 0.7278637886047363
epoch 1 train loss: 0.7457111477851868 val loss 0.6842456459999084
ep

 67%|██████▋   | 2/3 [04:31<02:15, 135.52s/it]

epoch 2 train loss: 7.531862735748291 val loss 6.596707820892334
epoch 2 train loss: 7.127125263214111 val loss 6.170927047729492
epoch 2 train loss: 5.237734317779541 val loss 3.980032444000244
epoch 2 train loss: 3.6068172454833984 val loss 2.3838143348693848
epoch 2 train loss: 2.3112380504608154 val loss 1.6114318370819092
epoch 2 train loss: 1.5837903022766113 val loss 1.3019189834594727
epoch 2 train loss: 1.297204852104187 val loss 1.203233242034912
epoch 2 train loss: 1.198825716972351 val loss 1.1696550846099854
epoch 2 train loss: 1.1708351373672485 val loss 1.1541869640350342
epoch 2 train loss: 0.9028104543685913 val loss 0.9276166558265686
epoch 2 train loss: 0.9291794300079346 val loss 0.9306899905204773
epoch 2 train loss: 0.9309723377227783 val loss 0.9296624064445496
epoch 2 train loss: 0.9256429076194763 val loss 0.9108467102050781
epoch 2 train loss: 0.9183791875839233 val loss 0.9005027413368225
epoch 2 train loss: 0.8904810547828674 val loss 0.8554084300994873
epoc

100%|██████████| 3/3 [06:45<00:00, 135.09s/it]


In [ ]:
torch.save(net.state_dict(), 'drive/MyDrive/Experimentos/NN/model_initial.pth')